In [10]:
import socket
import threading

class connection_mgr:
    
    def __init__(self):
        
        # Make shared variables thread-safe
        self.lock = threading.Lock()

        # Define the host and port to listen on
        self.host = '127.0.0.1'
        self.port = None
        
        # Initialize
        self.terminate = False
        self.received_data = None
        
        # Initialize a socket object
        self.server_socket = None

        # Shared variable to store received data
        self.received_data = None

        # Flag to indicate termination
        self.terminate = False
        
        # Initiate client socket
        self.client_socket = None
        
        # Manage which mode to use
        self.mode = None
        
    # Function to start server and wait for a connection
    def start_server(self, host = '127.0.0.1', port = 12345):
        
        if self.mode == 'client':
            print('connection_mgr.start_server: Already in client mode, terminating.')
            return False
        
        with self.lock:
            self.mode = 'server'
        
        # should add some checks here to be safe...
        self.host = host
        self.port = port
        
        # Create a socket object
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # Bind the socket to the host and port
        self.server_socket.bind((self.host, self.port))

        # Listen for incoming connections
        self.server_socket.listen(1)
        
        while not self.terminate:
            # Accept a client connection
            if not self.client_socket:
                try:
                    print('connection_mgr.start_server: Waiting for a new connection...')
                    self.client_socket, self.client_address = self.server_socket.accept()
                    print('connection_mgr.start_server: Connected by', self.client_address)

                    self.receive_data()

                    # Close client socket
                    print('connection_mgr.start_server: Client disconnected, cleaning up client socket.')
                    self.client_socket.close()
                    self.client_socket = None
                except Exception as ex:
                    print('connection_mgr.start_server: ' + str(ex))
                    if self.client_socket:
                        self.client_socket.close()
                        self.client_socket = None
                    self.terminate = True
                
        # Clean up server socket
        self.server_socket.close()
        self.server_socket = None
        
        with self.lock:
            self.mode = None
        
        print('connection_mgr.start_server: Terminated waiting for a new connection.')
        return True
    
    # function to start client and connect to server
    def start_client(self, host = '127.0.0.1', port = 12345):
        
        if self.mode == 'server':
            print('connection_mgr.start_client: Already in server mode, terminating.')
            return False
        
        with self.lock:
            self.mode = 'client'
        
        # Create a socket object
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Add logging
        log = logging.getLogger(__name__)

        # Define the server's host and port
        self.host = host
        self.port = port

        # Connect to the server
        self.client_socket.connect((self.host, self.port))
        print('connection_mgr.start_client: Connected to the server ', self.host, ' at port ', self.port)
        
    # function to stop the client
    def stop_client(self):
        
        if not (self.mode == 'client'):
            print('Not in client mode, can\'t stop client.')
            return False
        
        self.client_socket.close()
        
        with self.lock:
            self.mode = None
        
        return True
        

    # Function to handle receiving data from the client
    def receive_data(self):
        
        # Look for data until termination variable is set
        while not self.terminate:
            try:
                if self.client_socket:
                    data = self.client_socket.recv(1024).decode()
                    if not data:
                        break
                    print('connection_mgr.receive_data: Received:', data)
                    response = 'Hello, client! You sent me: ' + data
                    self.send_data(response.encode())
                    # Store received data in the shared variable
                    with self.lock:
                        self.received_data = data
            except socket.error:
                print('connection_mgr.receive_data: Encountered a socket error')
                break
                
        print('connection_mgr.receive_data: Connection was terminated.')
        
    # Function to send data to the client
    def send_data(self, message):
        if self.client_socket:
            try:
                self.client_socket.send(message)
                print('connection_mgr.send_data: successfully sent: ', message)
                return True
            except Exception as ex:
                print('connection_mgr.send_data: Encountered: ', str(ex))
                return False
        else:
            print('connection_mgr.send_data: Could not send data, client socket not connected.')
            return False
            

# Main code here:

connection = connection_mgr()

# Create a thread to wait for a connection
print('Main: Starting a new thread to initialize the connection')
connection_thread = threading.Thread(target=connection.start_server, args=())
connection_thread.start()

"""
# Create a thread to handle receiving data
print('Main: Starting a new thread to listen for data')
receive_thread = threading.Thread(target=connection.receive_data, args=())
receive_thread.start()
"""

# Main code execution
while True:
    # Access received data within the main loop
    with connection.lock:
        if connection.received_data:
            print('Main: Data received:', connection.received_data)
            # Reset the received data
            connection.received_data = None

    # Your other code here
    ...

    # Termination condition
    if len(input('Enter anything to terminate: ')) > 0:  # Add your termination condition here
        print('Main: Terminating connection.')
        connection.send_data('Terminating connection.'.encode())
        connection.terminate = True
        break

# Close the connection
try:
    if connection.client_socket:
        connection.client_socket.close()
    connection.server_socket.close()
except:
    pass

Main: Starting a new thread to initialize the connection
connection_mgr.start_server: Waiting for a new connection...


Enter anything to terminate:  h


Main: Terminating connection.
connection_mgr.send_data: Could not send data, client socket not connected.
connection_mgr.start_server: [WinError 10038] An operation was attempted on something that is not a socket
connection_mgr.start_server: Terminated waiting for a new connection.


In [9]:
# Close the connection
connection.client_socket.close()
connection.server_socket.close()